In [1]:
import pandas as pd
import os
import bs4
import json
import requests
from datetime import timedelta, date
from bs4 import BeautifulSoup

In [2]:
base_url = 'https://codigospostalessitecorecms.azurewebsites.net/Codigo-Postal/'
getMunicipality = 'wApi/getMunicipality'
getSettlement='wApi/getSettlement'

In [17]:
f = open('../output/Chiapas_houses.json',)
data = json.load(f)

In [29]:
type(data[0])

dict

In [30]:
try:
    for element in data:
        estado = element['Estado']
        ciudad = element['Ciudad'].upper()
        colonia= element['Colonia'].upper()
        
        first_page = requests.get(base_url)
        html = bs4.BeautifulSoup(first_page.text, "html5lib")
        select = html.find('select', attrs={'id':'cve_state'})
        options = select.findAll('option')
        
        for option in options:
            if option.text == estado:
                cve_estado = option['value']
                print(f'Estado {cve_estado}')
                
                payload = {"cve_state":cve_estado,\
                           "full":"true"}
                second_page = requests.post(base_url+getMunicipality, data=payload)
                html = bs4.BeautifulSoup(second_page.text, "html5lib")
                select = html.find('select', attrs={'id':'cve_municipality'})
                options = select.findAll('option')
                
                for option in options:
                    #print(f'{option.text}-{ciudad}')
                    if option.text.upper() == ciudad:
                        cve_municipality = option['value']
                        print(f'Ciudad {cve_municipality}')
                        
                        payload = {"cve_state":cve_estado,\
                                   "cve_municipality":cve_municipality}
                        third_page = requests.post(base_url+getMunicipality, data=payload)
                        html = bs4.BeautifulSoup(third_page.text, "html5lib")
                        select = html.find('select', attrs={'id':'cve_settlement'})
                        options = select.findAll('option')

                        for option in options:
                            #print(f'{option.text}-{colonia}')
                            if option.text.upper() == colonia:
                                cve_settlement = option['value']
                                print(f'Colonia {cve_settlement}')

                                payload = {"cve_state":cve_estado,\
                                           "cve_municipality":cve_municipality,\
                                           "cve_settlement":cve_settlement}
                                fourth_page = requests.post(base_url+getMunicipality, data=payload)
                                html = bs4.BeautifulSoup(fourth_page.text, "html5lib")
                                tr = html.find('tr', class_='pagedemo _current')
                                codigo_postal = tr.find('td').text
                                print(f'CP {codigo_postal}')
                                print('-------------')
                                element['cp'] = codigo_postal
                
except:
        print(element)

Estado 07
Ciudad 078
Colonia 0375
CP 29270
-------------
Estado 07
Ciudad 078
Colonia 0342
CP 29230
-------------
Estado 07
Ciudad 078
Colonia 0343
CP 29230
-------------


In [31]:
data

[{'url': 'https://casa.metroscubicos.com/MLM-924394115-en-venta-casa-nueva-_JM#position=1&search_layout=grid&type=item&tracking_id=3eca2dbc-a0ba-4293-9da2-86c3622a7ebd',
  'type_of_offer': 'sale',
  'type_of_prop': 'houses',
  'name': 'En Venta, Casa Nueva.',
  'price': 4150000,
  'currency': 'MXN',
  'Estado': 'Chiapas',
  'Ciudad': 'San Cristóbal De Las Casas',
  'Colonia': 'San Diego',
  'Superficie total': 175.0,
  'Superficie construida': 230.0,
  'Recamaras': 3.0,
  'Banos': 3.0,
  'Estacionamientos': 2.0,
  'Antiguedad': 0.0,
  'cp': '29270'},
 {'url': 'https://casa.metroscubicos.com/MLM-924438370-casa-en-venta-barrio-de-cuxtitali-_JM#position=2&search_layout=grid&type=item&tracking_id=3eca2dbc-a0ba-4293-9da2-86c3622a7ebd',
  'type_of_offer': 'sale',
  'type_of_prop': 'houses',
  'name': 'Casa En Venta, Barrio De Cuxtitali',
  'price': 300000,
  'currency': 'USD',
  'Estado': 'Chiapas',
  'Ciudad': 'San Cristóbal De Las Casas',
  'Colonia': 'Cuxtitali',
  'Superficie total': 610